In [ ]:
!pip install openai langchain faiss-cpu tiktoken

import openai
import faiss
import numpy as np
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.docstore.document import Document
from langchain.chains import RetrievalQA
import matplotlib.pyplot as plt
import pandas as pd
import os

openai.api_key = os.getenv("API-key-removed-for-safety")

In [ ]:
relevant_text = "The mitochondria generates ATP energy through cellular respiration. It is considered the powerhouse of the cell."
distractor_text = "Random historical fact: The moon landing occurred in 1969. "

def generate_long_doc(relevant, distractor, num_repeats=500, position="middle"):
    distractors = [distractor] * num_repeats
    if position == "start":
        content = [relevant] + distractors
    elif position == "end":
        content = distractors + [relevant]
    else:
        half = num_repeats // 2
        content = distractors[:half] + [relevant] + distractors[half:]
    return "\n".join(content)

long_document = generate_long_doc(relevant_text, distractor_text, num_repeats=400)

In [ ]:
def naive_chunking(text, chunk_size=1000):
    return [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

def recursive_chunking(text):
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    return splitter.split_text(text)

def create_rag_chain(text_chunks):
    docs = [Document(page_content=chunk) for chunk in text_chunks]
    embeddings = OpenAIEmbeddings()
    vectorstore = FAISS.from_documents(docs, embeddings)
    retriever = vectorstore.as_retriever(search_kwargs={"k": 5})
    llm = OpenAI(temperature=0, model_name="gpt-3.5-turbo")
    return RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

In [ ]:
def summarize_with_prompt(text, model="gpt-3.5-turbo"):
    llm = OpenAI(temperature=0, model_name=model)
    prompt = f"""Summarize the following text in 1-2 sentences. Focus only on the biologically relevant parts.

{text}
"""
    return llm(prompt)

In [ ]:
EVALUATION_PROMPT_TEMPLATE = """You will be given one summary written for an article. Your task is to rate the summary on one metric.

Evaluation Criteria:

{criteria}

Evaluation Steps:

{steps}

Example:

Source Text:

{document}

Summary:

{summary}

Evaluation Form (scores ONLY):

- {metric_name}
"""

# Metric 1: Relevance

RELEVANCY_SCORE_CRITERIA = """
Relevance(1-5) - selection of important content from the source. \
The summary should include only important information from the source document. \
Annotators were instructed to penalize summaries which contained redundancies and excess information.
"""

RELEVANCY_SCORE_STEPS = """
1. Read the summary and the source document carefully.
2. Compare the summary to the source document and identify the main points of the article.
3. Assess how well the summary covers the main points of the article, and how much irrelevant or redundant information it contains.
4. Assign a relevance score from 1 to 5.
"""

# Metric 2: Coherence

COHERENCE_SCORE_CRITERIA = """
Coherence(1-5) - the collective quality of all sentences. \
We align this dimension with the DUC quality question of structure and coherence \
whereby "the summary should be well-structured and well-organized. \
The summary should not just be a heap of related information, but should build from sentence to a\
coherent body of information about a topic."
"""

COHERENCE_SCORE_STEPS = """
1. Read the article carefully and identify the main topic and key points.
2. Read the summary and compare it to the article. Check if the summary covers the main topic and key points of the article,
and if it presents them in a clear and logical order.
3. Assign a score for coherence on a scale of 1 to 5, where 1 is the lowest and 5 is the highest based on the Evaluation Criteria.
"""

# Metric 3: Consistency

CONSISTENCY_SCORE_CRITERIA = """
Consistency(1-5) - the factual alignment between the summary and the summarized source. \
A factually consistent summary contains only statements that are entailed by the source document. \
Annotators were also asked to penalize summaries that contained hallucinated facts.
"""

CONSISTENCY_SCORE_STEPS = """
1. Read the article carefully and identify the main facts and details it presents.
2. Read the summary and compare it to the article. Check if the summary contains any factual errors that are not supported by the article.
3. Assign a score for consistency based on the Evaluation Criteria.
"""

# Metric 4: Fluency

FLUENCY_SCORE_CRITERIA = """
Fluency(1-3): the quality of the summary in terms of grammar, spelling, punctuation, word choice, and sentence structure.
1: Poor. The summary has many errors that make it hard to understand or sound unnatural.
2: Fair. The summary has some errors that affect the clarity or smoothness of the text, but the main points are still comprehensible.
3: Good. The summary has few or no errors and is easy to read and follow.
"""

FLUENCY_SCORE_STEPS = """
Read the summary and evaluate its fluency based on the given criteria. Assign a fluency score from 1 to 3.
"""

def get_geval_score(criteria, steps, document, summary, metric_name):
    prompt = EVALUATION_PROMPT_TEMPLATE.format(
        criteria=criteria,
        steps=steps,
        metric_name=metric_name,
        document=document,
        summary=summary,
    )
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0,
        max_tokens=5,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
    )
    return response["choices"][0]["message"]["content"]

In [ ]:
evaluation_metrics = {
    "Relevance": (RELEVANCY_SCORE_CRITERIA, RELEVANCY_SCORE_STEPS),
    "Coherence": (COHERENCE_SCORE_CRITERIA, COHERENCE_SCORE_STEPS),
    "Consistency": (CONSISTENCY_SCORE_CRITERIA, CONSISTENCY_SCORE_STEPS),
    "Fluency": (FLUENCY_SCORE_CRITERIA, FLUENCY_SCORE_STEPS),
}

In [ ]:
naive_chunks = naive_chunking(long_document)
rec_chunks = recursive_chunking(long_document)

naive_summary = summarize_with_prompt(naive_chunks[0])
rec_summary = summarize_with_prompt(rec_chunks[0])
rag_chain = create_rag_chain(rec_chunks)
rag_summary = rag_chain.run("Summarize the biological role of mitochondria.")

summaries = {
    "Naive": naive_summary,
    "Recursive": rec_summary,
    "RAG": rag_summary,
}

excerpt = long_document[:3000]

data = {"Evaluation Type": [], "Summary Type": [], "Score": []}

for eval_type, (criteria, steps) in evaluation_metrics.items():
    for summ_type, summary in summaries.items():
        data["Evaluation Type"].append(eval_type)
        data["Summary Type"].append(summ_type)
        result = get_geval_score(criteria, steps, excerpt, summary, eval_type)
        score_num = int(result.strip())
        data["Score"].append(score_num)

pivot_df = pd.DataFrame(data, index=None).pivot(
    index="Evaluation Type", columns="Summary Type", values="Score"
)
styled_pivot_df = pivot_df.style.highlight_max(axis=1)
display(styled_pivot_df)

In [ ]:
pivot_df.plot(kind='bar', figsize=(10, 6), title='G-Eval GPT-4 Scores for Context Strategies')
plt.ylabel("Score")
plt.xticks(rotation=0)
plt.ylim(0, 5.5)
plt.grid(True)
plt.show()